In [0]:
import pandas as pd
import numpy as np

In [0]:
nome_colunas = ['CD_MUNICIPIO', 'SG_UF', 'NM_REGIAO_SAUDE_ATUAL', 'NM_MUNICIPIO',
       'NM_REGIAO_SAUDE_ANTERIOR', 'NM_MUNICIPIOS_IN_37', 'ESTADO', 'REGIAO_PAIS']

In [0]:
df_regioes = pd.read_excel('BI_Regionais de saude.xlsx', sheet_name='Dados', skiprows=1, usecols="A:H", names=nome_colunas)

In [21]:
df_regioes.head()

,CD_MUNICIPIO,SG_UF,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS
0,251320,PB,10ª Região,Santa Cruz,X Núcleo Regional de Saúde,Santa Cruz,PARAÍBA,NORDESTE
1,250077,PB,10ª Região,Aparecida,X Núcleo Regional de Saúde,Aparecida,PARAÍBA,NORDESTE
2,251000,PB,10ª Região,Nazarezinho,X Núcleo Regional de Saúde,Nazarezinho,PARAÍBA,NORDESTE
3,251398,PB,10ª Região,São Francisco,X Núcleo Regional de Saúde,São Francisco,PARAÍBA,NORDESTE
4,251620,PB,10ª Região,Sousa,X Núcleo Regional de Saúde,Sousa,PARAÍBA,NORDESTE


In [22]:
df_regioes.shape

(5571, 8)

In [0]:
# verificar se tem regiões que abrangem mais de uma UF:
# df_regioes.groupby(['NM_REGIAO_SAUDE_ATUAL', 'SG_UF']).size()
df_unicos = df_regioes.drop_duplicates(subset=['NM_REGIAO_SAUDE_ATUAL', 'SG_UF'])


In [24]:
df_unicos.shape  #438, 8

(438, 8)

In [0]:
# Agrupa para selecionar Região de Saúde no DataFrame. Depois aplica o o metodo duplicated, para selecionar os efetivamente repetidos
linhas_duplicadas = df_regioes.groupby(['NM_REGIAO_SAUDE_ATUAL', 'SG_UF'], as_index=False).count()



In [47]:
linhas_duplicadas.shape

(438, 8)

In [49]:
# Seleciona todas as linhas duplicadas baseadas em uma lista de colunas. O parametro keep é para exibir todos os repetidos e não o último ou primeiro
linhas_duplicadas = linhas_duplicadas[linhas_duplicadas.duplicated(subset=['NM_REGIAO_SAUDE_ATUAL'], keep=False)]
linhas_duplicadas.shape  #(17, 8)
linhas_duplicadas
# Análise: Observando o campo NM_REGIAO_SAUDE_ATUAL e SG_UF, observa-se que: As regiões repetidas e que são de UF's contíguas São: Baixo Amazonas (AM e PA).
# O restante não são contíguas. Destaque para a Região de Saúde nomeada de "NORTE". Observando-se RJ e ES, que são contíguos, porém as respectivas Regiões situam-se no Norte de cada Estado.



,NM_REGIAO_SAUDE_ATUAL,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS
115,Baixo Amazonas,AM,5,5,5,5,5,5
116,Baixo Amazonas,PA,14,14,13,13,14,14
142,Central,ES,18,18,18,18,18,18
143,Central,GO,26,26,26,26,26,26
144,Central,RO,14,14,14,14,14,14
147,Centro Norte,MT,7,7,7,7,7,7
148,Centro Norte,RR,9,9,9,9,9,9
251,Metropolitana I,PA,5,5,5,5,5,5
252,Metropolitana I,RJ,12,12,12,12,12,12
253,Metropolitana II,PA,9,9,9,9,9,9


In [0]:
# Esta linha e para agilizar o groupby. Talvez desnecessaria. Lógica do do SAS
df_regioes.sort_values(by=['NM_REGIAO_SAUDE_ATUAL', 'SG_UF'], inplace=True)

In [0]:
lista1 = df_regioes.groupby(['NM_REGIAO_SAUDE_ATUAL', 'SG_UF'])['CD_MUNICIPIO'].apply(lambda x:list(set(x)))

In [0]:
lista1 = lista1.reset_index()

In [53]:
lista1.head()

,NM_REGIAO_SAUDE_ATUAL,SG_UF,CD_MUNICIPIO
0,10ª RS Cascavel,PR,"[410754, 410500, 410630, 411275, 411670, 41053..."
1,10ª Região,PB,"[250915, 251620, 251398, 251420, 251720, 25084..."
2,10ª Região Limoeiro do Norte,CE,"[230690, 230600, 230760, 231080, 231310, 23115..."
3,10ª Região de Saúde,AL,"[270240, 270330, 270500, 270642, 270580, 27071..."
4,11ª RS Campo Mourão,PR,"[412800, 410755, 411400, 410250, 410770, 41039..."


In [54]:
lista1.shape

(438, 3)

In [0]:
# Até aqui, para a região Norte, não tem Problemas
teste = lista1[lista1['NM_REGIAO_SAUDE_ATUAL']=='Norte']
# Para o caso da Região Baixo Amazonas que abrange 2 UF's, deve-se colocar a condição de agregar, para este caso, as duas UF's 
# teste = lista1[lista1['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas']

In [0]:
# teste.to_excel('teste2.xlsx')
lista1.to_excel('lista1.xlsx')

In [15]:
teste.shape

(2, 3)

In [16]:
teste.head()

,NM_REGIAO_SAUDE_ATUAL,SG_UF,CD_MUNICIPIO
115,Baixo Amazonas,AM,"[130050, 130340, 130290, 130068, 130300]"
116,Baixo Amazonas,PA,"[150145, 150050, 150530, 150565, 150600, 15047..."


In [83]:
teste['CD_MUNICIPIO'].str.len()

115     5
116    14
Name: CD_MUNICIPIO, dtype: int64

In [0]:
teste = lista1[lista1['NM_REGIAO_SAUDE_ATUAL']=='Baixo Amazonas']

In [86]:
teste['CD_MUNICIPIO'].str.len()

115     5
116    14
Name: CD_MUNICIPIO, dtype: int64

In [0]:
# verificar se tem regiões (ANTERIOR que abrangem mais de uma UF:
# df_regioes.groupby(['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF']).size()
df_unicos = df_regioes.drop_duplicates(subset=['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF'])


In [58]:
df_unicos.shape  #362, 8

(362, 8)

In [59]:
# Agrupa para selecionar Região de Saúde no DataFrame. Depois aplica o o metodo duplicated, para selecionar os efetivamente repetidos
linhas_duplicadas = df_regioes.groupby(['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF'], as_index=False).count()
linhas_duplicadas.shape # 358, 8

(358, 8)

In [60]:
# Seleciona todas as linhas duplicadas baseadas em uma lista de colunas. O parametro keep é para exibir todos os repetidos e não o último ou primeiro
linhas_duplicadas = linhas_duplicadas[linhas_duplicadas.duplicated(subset=['NM_REGIAO_SAUDE_ANTERIOR'], keep=False)]
linhas_duplicadas.shape  #(17, 8)
linhas_duplicadas
# Análise: Observando o campo NM_REGIAO_SAUDE_ANTERIOR e SG_UF, observa-se que: As regiões repetidas e que são de UF's contíguas São: Baixo Amazonas (AM e PA).
# O restante não são contíguas.


,NM_REGIAO_SAUDE_ANTERIOR,SG_UF,CD_MUNICIPIO,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS
26,Baixo Amazonas,AM,5,5,5,5,5,5
27,Baixo Amazonas,PA,9,9,9,9,9,9
187,Metropolitana,PA,5,5,5,5,5,5
188,Metropolitana,PR,29,29,29,29,29,29
211,Norte,GO,13,13,13,13,13,13
212,Norte,RJ,8,8,8,8,8,8


In [0]:
# =========================================

In [0]:
# Agora a lista para a Região de Saúde anterior
df_regioes.sort_values(by=['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF'], inplace=True)


In [0]:
lista2 = df_regioes.groupby(['NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF'])['CD_MUNICIPIO'].apply(lambda x:list(set(x)))

In [0]:
lista2 = lista2.reset_index()

In [65]:
lista2.shape

(358, 3)

In [0]:
lista2.to_excel('lista2.xlsx')

In [0]:
# =========================================
# OPERAÇÕES DE CONCATENAÇÃO DAS LISTAS DE MUNICIPIOS COM O DATAFRAME ORIGINAL

In [0]:
df_regioes_concat1 = pd.merge(df_regioes, lista1, how = 'left', left_on=['NM_REGIAO_SAUDE_ATUAL','SG_UF'], right_on=['NM_REGIAO_SAUDE_ATUAL','SG_UF'])

In [81]:
df_regioes_concat1.head()

,CD_MUNICIPIO_x,SG_UF,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,CD_MUNICIPIO_y
0,230655,CE,12ª Região Acaraú,Itarema,Acaraú,Itarema,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
1,230725,CE,12ª Região Acaraú,Jijoca de Jericoacoara,Acaraú,Jijoca de Jericoacoara,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
2,230780,CE,12ª Região Acaraú,Marco,Acaraú,Marco,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
3,230020,CE,12ª Região Acaraú,Acaraú,Acaraú,Acaraú,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."
4,230230,CE,12ª Região Acaraú,Bela Cruz,Acaraú,Bela Cruz,CEARÁ,NORDESTE,"[230020, 230725, 230890, 230230, 230425, 23078..."


In [0]:
teste = df_regioes_concat1[df_regioes_concat1['NM_REGIAO_SAUDE_ATUAL']=='Norte']

In [0]:
teste.to_excel('teste.xlsx')

In [0]:
df_regioes_concat1.rename(columns={'CD_MUNICIPIO_y':'lista_regiao_atual'}, inplace=True)

In [0]:
df_regioes_concat2 = pd.merge(df_regioes, lista2, how = 'left', left_on=['NM_REGIAO_SAUDE_ANTERIOR','SG_UF'], right_on=['NM_REGIAO_SAUDE_ANTERIOR','SG_UF'])

In [36]:
df_regioes_concat2.head()

,CD_MUNICIPIO_x,SG_UF_x,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,SG_UF_y,CD_MUNICIPIO_y
0,230655,CE,12ª Região Acaraú,Itarema,Acaraú,Itarema,CEARÁ,NORDESTE,CE,"[230020, 230725, 230890, 230230, 230425, 23078..."
1,230725,CE,12ª Região Acaraú,Jijoca de Jericoacoara,Acaraú,Jijoca de Jericoacoara,CEARÁ,NORDESTE,CE,"[230020, 230725, 230890, 230230, 230425, 23078..."
2,230780,CE,12ª Região Acaraú,Marco,Acaraú,Marco,CEARÁ,NORDESTE,CE,"[230020, 230725, 230890, 230230, 230425, 23078..."
3,230020,CE,12ª Região Acaraú,Acaraú,Acaraú,Acaraú,CEARÁ,NORDESTE,CE,"[230020, 230725, 230890, 230230, 230425, 23078..."
4,230230,CE,12ª Região Acaraú,Bela Cruz,Acaraú,Bela Cruz,CEARÁ,NORDESTE,CE,"[230020, 230725, 230890, 230230, 230425, 23078..."


In [0]:
df_regioes_concat2.rename(columns={'CD_MUNICIPIO_y':'lista_regiao_anterior'}, inplace=True)

In [0]:
colunas_mantidas = df_regioes_concat2.columns.difference(df_regioes_concat1.columns)

In [41]:
colunas_mantidas

Index(['lista_regiao_anterior'], dtype='object')

In [0]:
df_regioes_concat = pd.merge(df_regioes_concat1, df_regioes_concat2[colunas_mantidas], left_index=True, right_index=True, how='outer')

In [57]:
df_regioes_concat.head()

,CD_MUNICIPIO,SG_UF_x,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,SG_UF_y,lista_regiao_atual,lista_regiao_anterior,flag_atual
0,230655,CE,12ª Região Acaraú,Itarema,Acaraú,Itarema,CEARÁ,NORDESTE,CE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0
1,230725,CE,12ª Região Acaraú,Jijoca de Jericoacoara,Acaraú,Jijoca de Jericoacoara,CEARÁ,NORDESTE,CE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0
2,230780,CE,12ª Região Acaraú,Marco,Acaraú,Marco,CEARÁ,NORDESTE,CE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0
3,230020,CE,12ª Região Acaraú,Acaraú,Acaraú,Acaraú,CEARÁ,NORDESTE,CE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0
4,230230,CE,12ª Região Acaraú,Bela Cruz,Acaraú,Bela Cruz,CEARÁ,NORDESTE,CE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0


In [0]:
df_regioes_concat.rename(columns={'CD_MUNICIPIO_x':'CD_MUNICIPIO'}, inplace=True)

In [0]:
df_regioes_concat['flag_atual'] = np.where(df_regioes_concat['lista_regiao_atual'] == df_regioes_concat['lista_regiao_anterior'], 0, 1)

In [0]:
df_municipios = pd.read_excel('AR_BR_RG_UF_MES_MIC_MUN_2018.xls')

In [51]:
df_municipios.head()

,ID,CD_GCUF,NM_UF,NM_UF_SIGLA,CD_GCMUN,NM_MUN_2018,AR_MUN_2018,CD_MUNICIPIO_NDV
0,1.0,11.0,Rondônia,RO,1100015.0,ALTA FLORESTA D'OESTE,7067.025,110001
1,2.0,11.0,Rondônia,RO,1100023.0,ARIQUEMES,4426.571,110002
2,3.0,11.0,Rondônia,RO,1100031.0,CABIXI,1314.352,110003
3,4.0,11.0,Rondônia,RO,1100049.0,CACOAL,3792.892,110004
4,5.0,11.0,Rondônia,RO,1100056.0,CEREJEIRAS,2783.300,110005


In [0]:
df_municipios.dropna(axis=0, how='any', subset=['CD_GCMUN'], inplace=True)

In [0]:
df_municipios['CD_MUNICIPIO_NDV'] = df_municipios['CD_GCMUN'].astype(str).str[:6]

In [50]:
type(df_municipios['CD_MUNICIPIO_NDV'][0])

str

In [0]:
df_regioes_concat['CD_MUNICIPIO'] = df_regioes_concat['CD_MUNICIPIO'].astype(str)

In [0]:
df_diferenca = pd.merge(df_regioes_concat, df_municipios, how = 'outer', left_on='CD_MUNICIPIO', right_on='CD_MUNICIPIO_NDV')

In [0]:
df_diferenca_qgis = df_diferenca[df_diferenca['flag_atual']==1]

In [0]:
df_regioes_concat.rename(columns={'SG_UF_x':'SG_UF'}, inplace=True)

In [0]:
df_regioes_concat.to_excel('teste.xlsx')

In [55]:
df_regioes_concat.head()

,CD_MUNICIPIO,SG_UF_x,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,ESTADO,REGIAO_PAIS,SG_UF_y,lista_regiao_atual,lista_regiao_anterior,flag_atual
0,230655,CE,12ª Região Acaraú,Itarema,Acaraú,Itarema,CEARÁ,NORDESTE,CE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0
1,230725,CE,12ª Região Acaraú,Jijoca de Jericoacoara,Acaraú,Jijoca de Jericoacoara,CEARÁ,NORDESTE,CE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0
2,230780,CE,12ª Região Acaraú,Marco,Acaraú,Marco,CEARÁ,NORDESTE,CE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0
3,230020,CE,12ª Região Acaraú,Acaraú,Acaraú,Acaraú,CEARÁ,NORDESTE,CE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0
4,230230,CE,12ª Região Acaraú,Bela Cruz,Acaraú,Bela Cruz,CEARÁ,NORDESTE,CE,"[230020, 230725, 230890, 230230, 230425, 23078...","[230020, 230725, 230890, 230230, 230425, 23078...",0


In [0]:
resumo1 = df_regioes_concat[['SG_UF', 'NM_MUNICIPIO', 'NM_REGIAO_SAUDE_ATUAL']]

In [0]:
df_reg_mesmos_municipios = df_regioes_concat[df_regioes_concat['flag_atual']==0]

In [65]:
df_reg_mesmos_municipios.shape

(402, 12)

In [0]:
municipios_sem_mudar_regiao = df_reg_mesmos_municipios.shape[0]

In [70]:
municipios_sem_mudar_regiao

402

In [71]:
df_reg_mesmos_municipios.groupby('NM_REGIAO_SAUDE_ATUAL').count()

,CD_MUNICIPIO,SG_UF,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,Estado,Regiao,SG_UF_y,lista_regiao_atual,lista_regiao_anterior,flag_atual
NM_REGIAO_SAUDE_ATUAL,,,,,,,,,,,
10ª RS Cascavel,21,21,21,21,21,21,21,21,21,21,21
11ª RS Campo Mourão,21,21,21,21,21,21,21,21,21,21,21
12ª Região Acaraú,7,7,7,7,7,7,7,7,7,7,7
16ª RS Apucarana,17,17,17,17,17,17,17,17,17,17,17
16ª Região Camocim,1,1,1,1,1,1,1,1,1,1,1
19ª Região Brejo Santo,8,8,8,8,8,8,8,8,8,8,8
4ª Região Baturité,7,7,7,7,7,7,7,7,7,7,7
5ª Região Canindé,2,2,2,2,2,2,2,2,2,2,2
Afogados da Ingazeira,12,12,12,12,12,12,12,12,12,12,12


In [72]:
df_reg_mesmos_municipios.groupby('NM_REGIAO_SAUDE_ANTERIOR').count()

,CD_MUNICIPIO,SG_UF,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_MUNICIPIOS_IN_37,Estado,Regiao,SG_UF_y,lista_regiao_atual,lista_regiao_anterior,flag_atual
NM_REGIAO_SAUDE_ANTERIOR,,,,,,,,,,,
Acaraú,7,7,7,7,7,7,7,7,7,7,7
Afogados da Ingazeira,12,12,12,12,12,12,12,12,12,12,12
Alagoinhas,18,18,18,18,18,18,18,18,18,18,18
Alta Floresta,6,6,6,6,6,6,6,6,6,6,6
Apucarana,17,17,17,17,17,17,17,17,17,17,17
Araranguá,15,15,15,15,15,15,15,15,15,15,15
Arcoverde,13,13,13,13,13,13,13,13,13,13,13
Bacia Atlântico-Caetés,16,16,16,16,16,16,16,16,16,16,16
Baixada Cuiabana,11,11,11,11,11,11,11,11,11,11,11


In [0]:
resumo2 = df_reg_mesmos_municipios.groupby(['NM_REGIAO_SAUDE_ATUAL','NM_REGIAO_SAUDE_ANTERIOR', 'SG_UF']).count()['CD_MUNICIPIO'].reset_index()

In [0]:
resumo2.sort_values('CD_MUNICIPIO', ascending=False, inplace=True)

In [76]:
resumo2

,NM_REGIAO_SAUDE_ATUAL,NM_REGIAO_SAUDE_ANTERIOR,SG_UF,CD_MUNICIPIO
21,Caruaru,Caruaru,PE,28
28,Região 20 - Rota da Produção,CRS 15 Palmeira das Missões,RS,25
0,10ª RS Cascavel,Cascavel,PR,21
29,Região 21 - Sul,CRS 03 Pelotas,RS,21
27,Região 14 - Fronteira Noroeste,CRS 14 Santa Rosa,RS,21
1,11ª RS Campo Mourão,Campo Mourão,PR,21
26,Região 13 - Diversidade,CRS 17 Ijuí,RS,19
9,Alagoinhas,Alagoinhas,BA,18
3,16ª RS Apucarana,Apucarana,PR,17
33,Rio Caetés,Bacia Atlântico-Caetés,PA,16


In [0]:
df_regioes_concat.sort_values('SG_UF', ascending=True, inplace=True)

In [0]:
for row in df_regioes_concat.loc[df_regioes_concat['lista_regiao_anterior'].isnull(), 'lista_regiao_anterior'].index:
    df_regioes_concat.at[row, 'lista_regiao_anterior'] = []

In [80]:
df_regioes_concat[df_regioes_concat['lista_regiao_anterior'].isnull()]

,CD_MUNICIPIO,SG_UF,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,Estado,Regiao,SG_UF_y,lista_regiao_atual,lista_regiao_anterior,flag_atual


In [0]:
df_regioes_concat['flag_municipio_entrou'] = df_regioes_concat['lista_regiao_atual'].map(set) - df_regioes_concat['lista_regiao_anterior'].map(set)

In [0]:
df_regioes_concat['flag_municipio_saiu'] = df_regioes_concat['lista_regiao_anterior'].map(set) - df_regioes_concat['lista_regiao_atual'].map(set)

In [84]:
df_regioes_concat[df_regioes_concat['NM_REGIAO_SAUDE_ATUAL']=='Baixo Acre e Purus']

,CD_MUNICIPIO,SG_UF,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,Estado,Regiao,SG_UF_y,lista_regiao_atual,lista_regiao_anterior,flag_atual,flag_municipio_entrou,flag_municipio_saiu
1565,120032,AC,Baixo Acre e Purus,Jordão,Cruzeiro do Sul,Jordão,ACRE,NORTE,AC,"[120032, 120001, 120034, 120038, 120040, 12004...","[110080, 110020, 110013, 110026, 110060, 11009...",1,"{120032, 120001, 120034, 120038, 120040, 12004...","{110080, 110020, 110026, 110060, 110094, 11017..."
15,120001,AC,Baixo Acre e Purus,Acrelândia,Acre,Acrelândia,ACRE,NORTE,AC,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}
14,120013,AC,Baixo Acre e Purus,Bujari,Acre,Bujari,ACRE,NORTE,AC,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}
13,120017,AC,Baixo Acre e Purus,Capixaba,Acre,Capixaba,ACRE,NORTE,AC,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}
16,120080,AC,Baixo Acre e Purus,Porto Acre,Acre,Porto Acre,ACRE,NORTE,AC,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}
11,120038,AC,Baixo Acre e Purus,Plácido de Castro,Acre,Plácido de Castro,ACRE,NORTE,AC,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}
10,120040,AC,Baixo Acre e Purus,Rio Branco,Acre,Rio Branco,ACRE,NORTE,AC,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}
9,120043,AC,Baixo Acre e Purus,Santa Rosa do Purus,Acre,Santa Rosa do Purus,ACRE,NORTE,AC,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}
8,120050,AC,Baixo Acre e Purus,Sena Madureira,Acre,Sena Madureira,ACRE,NORTE,AC,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}
7,120045,AC,Baixo Acre e Purus,Senador Guiomard,Acre,Senador Guiomard,ACRE,NORTE,AC,"[120032, 120001, 120034, 120038, 120040, 12004...","[120001, 120034, 120038, 120040, 120043, 12004...",1,{120032},{}


In [100]:
df_regioes_concat[df_regioes_concat['NM_REGIAO_SAUDE_ATUAL']=='Norte']

,CD_MUNICIPIO,SG_UF,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,Estado,Regiao,SG_UF_y,lista_regiao_atual,lista_regiao_anterior,flag_atual,flag_municipio_entrou,flag_municipio_saiu,cont_municipio_entrou,cont_municipio_saiu
4472,320515,ES,Norte,Vila Pavão,SR Colatina,Vila Pavão,ESPÍRITO SANTO,SUDESTE,RJ,"[330240, 330500, 330475, 330093, 330415, 33048...","[310020, 316935, 310920, 311560, 314250, 31310...",1,"{330240, 330500, 330475, 330093, 330415, 33048...","{310020, 316935, 310920, 311560, 314250, 31310...",8,35
4471,320515,ES,Norte,Vila Pavão,SR Colatina,Vila Pavão,ESPÍRITO SANTO,SUDESTE,GO,"[521377, 521410, 522145, 522020, 520357, 52196...","[310020, 316935, 310920, 311560, 314250, 31310...",1,"{521377, 521410, 522145, 522020, 520357, 52196...","{310020, 316935, 310920, 311560, 314250, 31310...",13,35
4470,320515,ES,Norte,Vila Pavão,SR Colatina,Vila Pavão,ESPÍRITO SANTO,SUDESTE,ES,"[320160, 320515, 320100, 320390, 320360, 32042...","[310020, 316935, 310920, 311560, 314250, 31310...",1,"{320160, 320515, 320100, 320390, 320360, 32042...","{310020, 316935, 310920, 311560, 314250, 31310...",14,35
4469,320390,ES,Norte,Nova Venécia,SR Colatina,Nova Venécia,ESPÍRITO SANTO,SUDESTE,RJ,"[330240, 330500, 330475, 330093, 330415, 33048...","[210592, 211110, 210790, 210150, 210730, 21080...",1,"{330240, 330500, 330475, 330093, 330415, 33048...","{210592, 211110, 210150, 210790, 210730, 21080...",8,9
4468,320390,ES,Norte,Nova Venécia,SR Colatina,Nova Venécia,ESPÍRITO SANTO,SUDESTE,GO,"[521377, 521410, 522145, 522020, 520357, 52196...","[210592, 211110, 210790, 210150, 210730, 21080...",1,"{521377, 521410, 522145, 522020, 520357, 52196...","{210592, 211110, 210150, 210790, 210730, 21080...",13,9
4467,320390,ES,Norte,Nova Venécia,SR Colatina,Nova Venécia,ESPÍRITO SANTO,SUDESTE,ES,"[320160, 320515, 320100, 320390, 320360, 32042...","[210592, 211110, 210790, 210150, 210730, 21080...",1,"{320160, 320515, 320100, 320390, 320360, 32042...","{210592, 211110, 210150, 210790, 210730, 21080...",14,9
4473,320016,ES,Norte,Água Doce do Norte,SR Colatina,Água Doce do Norte,ESPÍRITO SANTO,SUDESTE,ES,"[320160, 320515, 320100, 320390, 320360, 32042...","[310020, 316935, 310920, 311560, 314250, 31310...",1,"{320160, 320515, 320100, 320390, 320360, 32042...","{310020, 316935, 310920, 311560, 314250, 31310...",14,35
4466,320090,ES,Norte,Barra de São Francisco,SR Colatina,Barra de São Francisco,ESPÍRITO SANTO,SUDESTE,RJ,"[330240, 330500, 330475, 330093, 330415, 33048...","[210592, 211110, 210790, 210150, 210730, 21080...",1,"{330240, 330500, 330475, 330093, 330415, 33048...","{210592, 211110, 210150, 210790, 210730, 21080...",8,9
4464,320090,ES,Norte,Barra de São Francisco,SR Colatina,Barra de São Francisco,ESPÍRITO SANTO,SUDESTE,ES,"[320160, 320515, 320100, 320390, 320360, 32042...","[210592, 211110, 210790, 210150, 210730, 21080...",1,"{320160, 320515, 320100, 320390, 320360, 32042...","{210592, 211110, 210150, 210790, 210730, 21080...",14,9
4465,320090,ES,Norte,Barra de São Francisco,SR Colatina,Barra de São Francisco,ESPÍRITO SANTO,SUDESTE,GO,"[521377, 521410, 522145, 522020, 520357, 52196...","[210592, 211110, 210790, 210150, 210730, 21080...",1,"{521377, 521410, 522145, 522020, 520357, 52196...","{210592, 211110, 210150, 210790, 210730, 21080...",13,9


In [0]:
df_regioes_concat['cont_municipio_entrou'] = [len(x) for x in df_regioes_concat['flag_municipio_entrou']]

In [0]:
df_regioes_concat['cont_municipio_saiu'] = [len(x) for x in df_regioes_concat['flag_municipio_saiu']]

In [87]:
df_regioes_concat[(df_regioes_concat['flag_atual']==0)&(df_regioes_concat['cont_municipio_entrou']>1)]

,CD_MUNICIPIO,SG_UF,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,Estado,Regiao,SG_UF_y,lista_regiao_atual,lista_regiao_anterior,flag_atual,flag_municipio_entrou,flag_municipio_saiu,cont_municipio_entrou,cont_municipio_saiu


In [88]:
df_regioes_concat[(df_regioes_concat['flag_atual']==0)&(df_regioes_concat['cont_municipio_saiu']>1)]

,CD_MUNICIPIO,SG_UF,NM_REGIAO_SAUDE_ATUAL,NM_MUNICIPIO,NM_REGIAO_SAUDE_ANTERIOR,NM_MUNICIPIOS_IN_37,Estado,Regiao,SG_UF_y,lista_regiao_atual,lista_regiao_anterior,flag_atual,flag_municipio_entrou,flag_municipio_saiu,cont_municipio_entrou,cont_municipio_saiu


In [0]:
df_estatistica_entrada = df_regioes_concat[(df_regioes_concat['flag_atual']==1)&(df_regioes_concat['cont_municipio_entrou']!=0)]

In [91]:
df_estatistica_entrada.shape

(4983, 16)

In [0]:
df_result_atual = df_estatistica_entrada.groupby(['SG_UF','NM_REGIAO_SAUDE_ATUAL'])['cont_municipio_entrou', 'cont_municipio_saiu'].agg(['count','min','max','mean']).reset_index()

In [93]:
df_result_atual

SG_UF             NM_REGIAO_SAUDE_ATUAL  ... cont_municipio_saiu           
                                             ...                 max       mean
0      AC                         Alto Acre  ...                   8   8.000000
1      AC                Baixo Acre e Purus  ...                  15   1.363636
2      AC           Juruá e Tarauacá/Envira  ...                  34  25.857143
3      AL               10ª Região de Saúde  ...                   9   9.000000
4      AL                1ª Região de Saúde  ...                  14   9.166667
5      AL                2ª Região de Saúde  ...                  10  10.000000
6      AL                3ª Região de Saúde  ...                  13  13.000000
7      AL                4ª Região de Saúde  ...                  16  14.444444
8      AL                5ª Região de Saúde  ...                  14  14.000000
9      AL                6ª Região de Saúde  ...                   9   6.750000
10     AL                7ª Região de Saúde  ...                  16  14.705882
11     AL                8ª Região de Saúde  ...                  14  14.000000
12     AL                9ª Região de Saúde  ...                  16  13.428571
13     AM                     Alto Solimões  ...                  10   2.222222
14     AM                    Baixo Amazonas  ...                   1   1.000000
15     AM  Manaus, Entorno e Alto Rio Negro  ...                  14  10.000000
16     AM                    Médio Amazonas  ...                   7   7.000000
17     AM                    Regional Juruá  ...                  49  37.666667
18     AM              Rio Negro e Solimões  ...                  19  18.000000
19     AM                         Triângulo  ...                  49  49.000000
20     AP                      Área Central  ...                  25  25.000000
21     AP                        Área Norte  ...                  25  25.000000
22     AP                     Área Sudoeste  ...                  25  25.000000
23     BA                         Barreiras  ...                  10  10.000000
24     BA                           Brumado  ...                   9   0.571429
25     BA                          Camaçari  ...                  48  40.000000
26     BA                    Cruz das Almas  ...                  15  15.000000
27     BA                  Feira de Santana  ...                  27  21.464286
28     BA                          Guanambi  ...                  28  14.190476
29     BA                         Ibotirama  ...                  24  24.000000
..    ...                               ...  ...                 ...        ...
363    SP                           Limeira  ...                  54  18.750000
364    SP                     Litoral Norte  ...                  21  21.000000
365    SP                        Mananciais  ...                  27  17.375000
366    SP                       Mantiqueira  ...                  28  23.500000
367    SP                           Marília  ...                  29  16.368421
368    SP                          Ourinhos  ...                  29  16.384615
369    SP                        Piracicaba  ...                  54  54.000000
370    SP            Pontal do Paranapanema  ...                  24  24.000000
371    SP  Região Metropolitana de Campinas  ...                  25  25.000000
372    SP                         Rio Claro  ...                  54  54.000000
373    SP                         Rio Pardo  ...                  22  22.000000
374    SP             Rota dos Bandeirantes  ...                  27  27.000000
375    SP                   Santa Fé do Sul  ...                  42  42.000000
376    SP                          Sorocaba  ...                  26  22.500000
377    SP             São José do Rio Preto  ...                  42  42.000000
378    SP                         São Paulo  ...                  27  27.000000
379    SP                      Três Colinas  ...                  21  18.000000
380 

In [0]:
df_result_anterior = df_estatistica_entrada.groupby(['SG_UF','NM_REGIAO_SAUDE_ANTERIOR'])['cont_municipio_entrou', 'cont_municipio_saiu'].agg(['count','min','max','mean']).reset_index()

In [95]:
df_result_anterior.head()

SG_UF NM_REGIAO_SAUDE_ANTERIOR cont_municipio_entrou  ... cont_municipio_saiu               
                                                 count  ...                 min max       mean
0    AC                     Acre                    10  ...                   0   0   0.000000
1    AC                Brasiléia                     1  ...                   8   8   8.000000
2    AC          Cruzeiro do Sul                     8  ...                  15  34  24.500000
3    AL                Região 01                    27  ...                   8  14  10.518519
4    AL                Região 02                    15  ...                   6  14  10.133333

[5 rows x 10 columns]

In [0]:
df_result_atual.to_excel('resultado_atual.xlsx')

In [0]:
df_result_anterior.to_excel('resultado_anterior.xlsx')

In [0]:
df_estatistica_entrada.to_excel('resumo_geral_movimentacao.xlsx')